In [8]:
from transformers import MarianMTModel, MarianTokenizer

# Download the pretrained model and tokenizer for French to English translation
model_name = "Helsinki-NLP/opus-mt-en-fr"
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

In [5]:
def translate_text(text, model, tokenizer):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt")

    # Translate the input text
    translated = model.generate(**inputs)

    # Decode the translated output
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

input_text = "Hello, how are you?"
translated_text = translate_text(input_text, model, tokenizer)
print(translated_text)


Hello, how are you?


In [6]:
def translate_batch(texts, model, tokenizer):
    # Tokenize the input texts
    inputs = tokenizer(texts, return_tensors="pt", padding=True)

    # Translate the input texts
    translated = model.generate(**inputs)

    # Decode the translated outputs
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return translated_texts

input_texts = ["Bonjour, comment ça va?", "Quel est ton nom?"]
translated_texts = translate_batch(input_texts, model, tokenizer)
print(translated_texts)


['Hello, how are you?', "What's your name?"]


In [9]:
def chunk_text(text, max_chunk_size=512):
    chunks = []
    current_chunk = ""
    
    for token in text.split():
        if len(current_chunk) + len(token) + 1 <= max_chunk_size:
            current_chunk += token + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = token + " "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks


In [12]:
def translate_chunks_in_batches(chunks, batch_size=4):
    translated_texts = []

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        batch_input = " ".join(batch)
        translated_batch =translate_batch(batch_input,model, tokenizer)
        translated_texts.extend(translated_batch.split("\n"))

    return translated_texts


In [13]:
def en_fr_translate_long_text(text, max_chunk_size=512, batch_size=4):
    chunks = chunk_text(text, max_chunk_size)
    translated_chunks = translate_chunks_in_batches(chunks, batch_size)
    translated_text = " ".join(translated_chunks)
    return translated_text


In [14]:
long_text = "Your very long input text..."
translated_text = en_fr_translate_long_text(long_text)
print(translated_text)


AttributeError: 'list' object has no attribute 'split'